In [ ]:
#This is an example script for the standard KFO single-cell/single-nuclei data analysis. 
#This specific example is ran on 10x single-nuclei data from glom prep.

## Set seed for replicability
set.seed(1234)

##Load libraries
library(Seurat)
library(Matrix)
library(tidyverse)
library(data.table)
library(celda)
library(scater)
library(scDblFinder)
library(gridExtra)
library(writexl)
library(gprofiler2)
library(decontX)

##Folders

dir= "/data/public/kbohl1/Mai13_TXM04_Michael/cellranger"
seu_path <- "./data/Seurat Objects/"
fig_out <- "./figures/"
data_out <- "./data/"
res_out  <- "./results/"
dir.create(fig_out)
dir.create(data_out)
dir.create(res_out)
dir.create(seu_path)

#Note: In the comments, library and cell are used interchangeably, even though, especially in droplet-based data, they are different.
#While a cell barcode should correspond to one cell, we have to correct for empty droplets and doublets.

##Step 1: Loading the data and removing ambient RNA
##The following custom function loads data from CellRanger.

#' Load in data from 10X
#'
#' Enables easy loading of sparse data matrices provided by 10X genomics.
#'
#' @param data.dir Directory containing the matrix.mtx, genes.tsv (or features.tsv), and barcodes.tsv
#' files provided by 10X. A vector or named vector can be given in order to load
#' several data directories. If a named vector is given, the cell barcode names
#' will be prefixed with the name.
#' @param gene.column Specify which column of genes.tsv or features.tsv to use for gene names; default is 2
#' @param cell.column Specify which column of barcodes.tsv to use for cell names; default is 1
#' @param unique.features Make feature names unique (default TRUE)
#' @param strip.suffix Remove trailing "-1" if present in all cell barcodes.
#'
#' @return If features.csv indicates the data has multiple data types, a list
#'   containing a sparse matrix of the data from each type will be returned.
#'   Otherwise a sparse matrix containing the expression data will be returned.
#'
#' @importFrom Matrix readMM
#' @importFrom utils read.delim
#'
#' @export
#' @concept preprocessing
#'
#' @examples
#' \dontrun{
#' # For output from CellRanger < 3.0
#' data_dir <- 'path/to/data/directory'
#' list.files(data_dir) # Should show barcodes.tsv, genes.tsv, and matrix.mtx
#' expression_matrix <- Read10X(data.dir = data_dir)
#' seurat_object = CreateSeuratObject(counts = expression_matrix)
#'
#' # For output from CellRanger >= 3.0 with multiple data types
#' data_dir <- 'path/to/data/directory'
#' list.files(data_dir) # Should show barcodes.tsv.gz, features.tsv.gz, and matrix.mtx.gz
#' data <- Read10X(data.dir = data_dir)
#' seurat_object = CreateSeuratObject(counts = data$`Gene Expression`)
#' seurat_object[['Protein']] = CreateAssayObject(counts = data$`Antibody Capture`)
#' }
#'
Read10X <- function(
    data.dir = NULL,
    gene.column = 2,
    cell.column = 1,
    unique.features = TRUE,
    strip.suffix = FALSE, 
    zipped=F 
) 
{
  full.data <- list()
  for (i in seq_along(along.with = data.dir)) {
    run <- data.dir[i]
    if (!dir.exists(paths = run)) {
      stop("Directory provided does not exist")
    }
    barcode.loc <- file.path(run, 'barcodes.tsv')
    gene.loc <- file.path(run, 'genes.tsv')
    features.loc <- file.path(run, 'features.tsv')
    matrix.loc <- file.path(run, 'matrix.mtx')
    # Flag to indicate if this data is from CellRanger >= 3.0
    pre_ver_3 <- file.exists(gene.loc)
    if (!pre_ver_3 & zipped==T ) {
      addgz <- function(s) {
        return(paste0(s, ".gz"))
      }
      barcode.loc <- addgz(s = barcode.loc)
      matrix.loc <- addgz(s = matrix.loc)
      features.loc <- addgz(s = features.loc)
    }
    if (!file.exists(barcode.loc)) {
      stop("Barcode file missing. Expecting ", basename(path = barcode.loc))
    }
    if (!pre_ver_3 && !file.exists(features.loc) ) {
      stop("Gene name or features file missing. Expecting ", basename(path = features.loc))
    }
    if (!file.exists(matrix.loc)) {
      stop("Expression matrix file missing. Expecting ", basename(path = matrix.loc))
    }
    data <- readMM(file = matrix.loc)
    cell.barcodes <- read.table(file = barcode.loc, header = FALSE, sep = '\t', row.names = NULL)
    if (ncol(x = cell.barcodes) > 1) {
      cell.names <- cell.barcodes[, cell.column]
    } else {
      cell.names <- readLines(con = barcode.loc)
    }
    if (all(grepl(pattern = "\\-1$", x = cell.names)) & strip.suffix) {
      cell.names <- as.vector(x = as.character(x = sapply(
        X = cell.names,
        FUN = ExtractField,
        field = 1,
        delim = "-"
      )))
    }
    if (is.null(x = names(x = data.dir))) {
      if (i < 2) {
        colnames(x = data) <- cell.names
      } else {
        colnames(x = data) <- paste0(i, "_", cell.names)
      }
    } else {
      colnames(x = data) <- paste0(names(x = data.dir)[i], "_", cell.names)
    }
    feature.names <- read.delim(
      file = ifelse(test = pre_ver_3, yes = gene.loc, no = features.loc),
      header = FALSE,
      stringsAsFactors = FALSE
    )
    if (any(is.na(x = feature.names[, gene.column]))) {
      warning(
        'Some features names are NA. Replacing NA names with ID from the opposite column requested',
        call. = FALSE,
        immediate. = TRUE
      )
      na.features <- which(x = is.na(x = feature.names[, gene.column]))
      replacement.column <- ifelse(test = gene.column == 2, yes = 1, no = 2)
      feature.names[na.features, gene.column] <- feature.names[na.features, replacement.column]
    }
    if (unique.features) {
      fcols = ncol(x = feature.names)
      if (fcols < gene.column) {
        stop(paste0("gene.column was set to ", gene.column,
                    " but feature.tsv.gz (or genes.tsv) only has ", fcols, " columns.",
                    " Try setting the gene.column argument to a value <= to ", fcols, "."))
      }
      rownames(x = data) <- make.unique(names = feature.names[, gene.column])
    }
    # In cell ranger 3.0, a third column specifying the type of data was added
    # and we will return each type of data as a separate matrix
    if (ncol(x = feature.names) > 2) {
      data_types <- factor(x = feature.names$V3)
      lvls <- levels(x = data_types)
      if (length(x = lvls) > 1 && length(x = full.data) == 0) {
        message("10X data contains more than one type and is being returned as a list containing matrices of each type.")
      }
      expr_name <- "Gene Expression"
      if (expr_name %in% lvls) { # Return Gene Expression first
        lvls <- c(expr_name, lvls[-which(x = lvls == expr_name)])
      }
      data <- lapply(
        X = lvls,
        FUN = function(l) {
          return(data[data_types == l, , drop = FALSE])
        }
      )
      names(x = data) <- lvls
    } else{
      data <- list(data)
    }
    full.data[[length(x = full.data) + 1]] <- data
  }
  # Combine all the data from different directories into one big matrix, note this
  # assumes that all data directories essentially have the same features files
  list_of_data <- list()
  for (j in 1:length(x = full.data[[1]])) {
    list_of_data[[j]] <- do.call(cbind, lapply(X = full.data, FUN = `[[`, j))
    # Fix for Issue #913
    list_of_data[[j]] <- as(object = list_of_data[[j]], Class = "dgCMatrix")
  }
  names(x = list_of_data) <- names(x = full.data[[1]])
  # If multiple features, will return a list, otherwise
  # a matrix.
  if (length(x = list_of_data) == 1) {
    return(list_of_data[[1]])
  } else {
    return(list_of_data)
  }
}

##List 10x folders. This step can be different, depending on your data.
lldir <- list.dirs(dir, recursive = F)

##Get vector of sample names. Here we get them from the folder names, but it can be done manually, from annotation files etc.
sample_names <- basename(lldir)

## Run DecontX on each individual mouse dataset. Ambient RNA removal is optional but empirically it improves our dataset quality.

for(i in 1:length(sample_names)){
  print(sample_names[i])
#Read filtered data
  sce <- Read10X(data.dir = paste0(lldir[i] , "/outs/filtered_feature_bc_matrix/"),  gene.column =2, zipped=T)
  sce<-sce$`Gene Expression` #If multiome.
#Read raw data
  sce.raw <- Read10X(data.dir = paste0(lldir[i] , "/outs/raw_feature_bc_matrix/"), gene.column =2, zipped=T)
  sce.raw<-sce.raw$`Gene Expression`  #If multiome.
#Use decontX to remove ambient RNA  
  sce.decont <- decontX(sce, background = sce.raw)
#Save file after ambient RNA removal
  saveRDS(sce.decont , paste0(seu_path , "filtered_decontX.",sample_names[i], ".rda"))
  rm(sce, sce.decont, sce.raw)
  print(" ")
}

##Create a list of Seurat objects including all data.

seuList <- lapply(seq(lldir), function(i){
  print(i)
  # Read counts after ambient RNA removal
  cmat.decontX <- readRDS(paste0(data_out, "filtered_decontX.", sample_names[i], ".rda"))
  # Create seurat object
  ssc.decontX <- CreateSeuratObject(counts = round(cmat.decontX$decontXcounts))
  return(ssc.decontX)
})

##Merge all samples in one object

seu <- merge(seuList[[1]] , y= seuList[2:length(seuList)] ,
              add.cell.ids= sample_names)

##Add metadata 

seu$sample<-sub( "_.*" , "" , colnames(seu)) #Sample labels

#More metadata columns can be added if relevant to the project(e.g. age, technical batch, condition, sex)

##Step :Doublet Identification

##Introduction

#Doublets, defined as two cells sequenced under the same cellular barcode (e.g., being captured in the same droplet), are fairly frequent in scRNAseq datasets,
#with estimates ranging from 1 to 10% depending on the platform and cell concentration used. While doublets of the same cell type are relatively innocuous for 
#most downstream applications due to their conservation of the relative expression between genes, doublets formed from different cell types or states are likely 
#to be misclassified and could potentially distort downstream analysis.

#In some cases, doublets can be identified through their unusually high number of reads and detected features, but this is not always the case.
#Therefore, a number of methods were developed to identify doublets, in most cases by comparing each cell to artificially created doublets.
#In this script, we use scDblFinder.
#Alternatives: DoubletFinder, doubletCell, scds

#Convert to SingleCellExperiment
seu_sce <- as.SingleCellExperiment(seu)

#Run scDblFinder
seu_sce <- scDblFinder(seu_sce, samples =  "sample", verbose = TRUE)

# show amount of doublets identified per sample
doublet_table <-  table(truth =  seu_sce$sample, call = seu_sce$scDblFinder.class)
#Calculate percentage
doublet_table <- as.data.table(doublet_table)
doublet_table <- pivot_wider(doublet_table, names_from = call, values_from = N)
doublet_table$doublet_percentage <- doublet_table$doublet/(doublet_table$singlet+doublet_table$doublet)
#Transform back to Seurat
seu <- as.Seurat(seu_sce, project = "orig.ident")

#Visualize doublet score distribution
data <- data.frame(seu$scDblFinder.score, seu$scDblFinder.class)
singlet.df <- data[data$seu.scDblFinder.class == "singlet",]
doublet.df <- data[data$seu.scDblFinder.class == "doublet",]

ggplot(data = singlet.df, aes(x = singlet.df$seu.scDblFinder.score)) + geom_histogram()
ggplot(data = doublet.df, aes(x = doublet.df$seu.scDblFinder.score)) + geom_histogram()

rm(seu_sce)

# Save post Dbl identification Seurat
saveRDS(seu, paste0(seu_path,  "seu_dbl.rds"))

#Step 3: Quality control

##3.1 Introduction

#Low quality cells can arise for multiple reasons, including cell damage and failure in library preparation.
#Their characteristics include low RNA counts, few expressed genes and high mitochondrial/ribosomal RNA percentage.
#These cells need to be removed from further analysis because:
  #1)They tend to cluster together and form distinct cell subpopulations, even if they belong to different cell types.
  #2)They reduce the efficiency of dimensional reduction, since they affect the differences captured by the first principal components.
  #3)They affect differential expression analysis (inclusion of mitochondrial genes, overrepresentation of contaminating transcripts)
#The removal of these cells is commonly referred to as quality control.

##3.2 Metrics

#To identify low-quality cells we use:
  #a)The library size. Cells with low library size are of low quality, since most of their RNA has been lost during one point of the preparations.
  #b)The number of expressed genes.
  #c)The proportion of reads mapped to the mitochondrial genome. In single-cell data, high proportions are an indication of damaged cells. In single-nuclei data, 
  #high proportions are indicative of unsuccessful cytoplasm stripping.

#The first two QC metrics can be found in the metadata of the Seurat object.
summary(seu@meta.data$nCount_RNA)   #Number of RNA molecules per cell
summary(seu@meta.data$nFeature_RNA) #Number of genes per cell

#The third can be calculated.
seu <- PercentageFeatureSet(seu, "^mt-", col.name = "percent_mito") #If mice
seu <- PercentageFeatureSet(seu, "^MT-", col.name = "percent_mito") #If human
summary(seu@meta.data$percent_mito)

#An important assumption is that the metrics are independent of the biological state of the cell.
#In certain datasets, there can be high variation in RNA content or mitochondrial expression in different cell types.
#In that case, caution should be exercised to prevent their loss.

#From experience, removal of ribosomal genes robustly reduced the quality of the clustering, suggesting that they represent 
#real biological differences between subpopulations.

##3.3 Low quality cell identification

##a)Fixed thresholds. An arbitrary fixed quality threshold is set for our metrics. Any cell outside of these thresholds is low quality.
##Advantages:    Simple
##Disadvantages: Different thresholds for each protocol and biological system. Even with a lot of experience, technical issues can have a big effect on the qc metrics distribution.
##Example:
qc.rna<-seu@meta.data$nCount_RNA<300
qc.gene<-seu@meta.data$nFeature_RNA<200
qc.mito<-seu@meta.data$percent_mito<0.05
discard <- qc.rna | qc.gene | qc.mito
#Summarize the number of discarded cells per reason.
DataFrame(nCount=sum(qc.rna), nFeature=sum(qc.gene),percent_mito=sum(qc.mito,na.rm=T), total=sum(discard,na.rm=T))

##b)Adaptive thresholds. Under the assumption that most of our cells are high quality, we remove outliers (more than 3 MADs from the median)
##Advantages:    The majority of the cells are not removed. Less arbitrary than approach (a).
##Disadvantages: An important risk of excluding cells on the basis of their distance from the whole distribution of cells on some properties (e.g., library size) 
##               is that these properties tend to have different distributions across subpopulations.
##               This can lead to strong biases against certain subpopulations .
##               Additionally, the underlying assumption of a high-quality majority may not always be appropriate

qc.rna<-isOutlier(seu@meta.data$nCount_RNA)
qc.gene<-isOutlier(seu@meta.data$nFeature_RNA)
qc.mito<-isOutlier(seu@meta.data$percent_mito)
discard <- qc.rna | qc.gene | qc.mito
#Summarize the number of discarded cells per reason.
DataFrame(nCount=sum(qc.rna), nFeature=sum(qc.gene),percent_mito=sum(qc.mito,na.rm=T), total=sum(discard,na.rm=T))

##c)Based on diagnostic plots.
##Create correlation and feature plots of the QC metrics. Remove outliers based on the plots.
##Advantages:Intuitive, allows better judgment of whether adaptive thresholds are appropriate, allows understanding of distribution.
##Disadvantages: Can be ill-defined/subjective.
##Diagnostic plots should be created regardless. 

p1 <- FeatureScatter(seu, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +
  geom_vline(xintercept = 300, color = "red") + geom_hline(yintercept = 8000, color = "red")
p2 <- FeatureScatter(seu, feature1 = "nCount_RNA", feature2 = "percent_mito") + 
  geom_vline(xintercept = 300, color = "red") + 
  geom_hline(yintercept = 1, color = "red")
cowplot::plot_grid(p1,p2, ncol = 2)
p3 <- VlnPlot(seu, features = "nCount_RNA", pt.size = FALSE)
p4 <- VlnPlot(seu, features = "nFeature_RNA", pt.size = FALSE)
p5 <- VlnPlot(seu, features = "percent_mito", pt.size = FALSE)
cowplot::plot_grid(p3,p4,p5, ncol = 3)
p6 <- VlnPlot(seu, features = "nCount_RNA", pt.size = FALSE,group.by = "sample")
p7 <- VlnPlot(seu, features = "nFeature_RNA", pt.size = FALSE,group.by = "sample")
p8 <- VlnPlot(seu, features = "percent_mito", pt.size = FALSE,group.by = "sample")
cowplot::plot_grid(p6,p7,p8, ncol = 3)

##3.4 Low quality cell filtering
##Low quality cells can either be marked or removed.
##Marking them defers the final decision until clustering and dimensional reduction, which can allow better judgment of which criteria 
##to use. The downside is that it shifts the burden of QC to the manual interpretation of the clusters, which is already 
##a major bottleneck in scRNA-seq data analysis. Retention of low-quality cells also compromises the accuracy of the 
##variance modelling

##For standard analysis, removal is highly recommended.
seu <- subset(seu, subset = scDblFinder.class=="singlet"&scDblFinder.score<0.5)
seu <- subset(seu, subset = nFeature_RNA > 200 & nCount_RNA> 500 & percent_mito < 1& nFeature_RNA<50000)


#Step 4: Normalization

##4.1 Introduction
##Technical differences in cDNA capture or PCR amplification efficiency create differences in sequencing coverage between libraries.
##Normalization aims to remove these differences so that differential expression/heterogeneity are driven by biological factors.

##4.2 Methods
##Two main approaches:
##a)Library size normalization and log-transformation.
seu<-NormalizeData(seu)
##b)sctransform
seu<-SCTransform(seu)

##Step 5: Feature selection and scaling

seu <- seu %>%
  FindVariableFeatures(selection.method = "vst", nfeatures = 2000) %>%
  ScaleData(vars.to.regress = "percent_mito") #Not necessary if sctransform was used for normalization

##Step 6: Dimensional reduction and clustering

#PCA
seu<-RunPCA(seu)
DimPlot(seu)
#Elbow plot
ElbowPlot(seu)
#UMAP and clustering
seu <- seu %>% 
  FindNeighbors(dims = 1:20) %>% #number of dimensions is arbitrary but elbow plot can help with the determination
  FindClusters(resolution = 0.2) %>% #the package clustree can be used to find the optimal clustering resolution
  RunUMAP(dims = 1:20)
DimPlot(seu)

##Step 7: Cell type identification

##Find cluster markers
seu.cluster.markers <- FindAllMarkers(seu, 
                                      logfc.threshold = 0.25,
                                      min.pct = 0.2,
                                      only.pos = TRUE)
cluster_identifier <- seu.cluster.markers %>%
  group_by(cluster) %>%
  slice_max(n = 5, order_by = avg_log2FC)

##Manual annotation
# Visualization of DE genes for cluster identification
DotPlot(seu, features = c("Nphs1", "Nphs2", "Thsd7a", "Itga8", "Gata3", "Flt1", "Pecam1", "Ptprc", "Itgax", "Slc34a1", "Slc12a3", "Slc4a9"))
DotPlot(seu, features = subset(cluster_identifier, subset = cluster == 1)$gene)
DotPlot(seu, features = cluster_identifier$gene)+ theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

# check qc metrics in different clusters
VlnPlot(seu, features = "scDblFinder.score", pt.size = F)

## Compare manual annotation with PanglaoDB markers
# AddModuleScore with Panglao markers
panglao_markers <- fread("./data/panglao_celltype_markers/panglaodb_markers.tsv")
colnames(panglao_markers)[3] <- "celltype"
# Exclude exclusively human markers
mm_panglao_markers <- panglao_markers[species != "Hs",]
# Convert gene symbols to mouse with function from nichenetr package
library(nichenetr)
mm_panglao_markers$mouse_symbol <- convert_human_to_mouse_symbols(mm_panglao_markers$`official gene symbol`)
# Exclude NAs (results from conversion to mouse; not 1:1 ortholog for every human gene available)
mm_panglao_markers <- mm_panglao_markers[is.na(mm_panglao_markers$mouse_symbol) == FALSE, ]

# Subset for kidney cell types
kidney_markers <- mm_panglao_markers[organ == "Kidney",]

for(type in unique(kidney_markers$celltype)){
  tmp_markers <- mm_panglao_markers[celltype == type,]
  seu <- AddModuleScore(seu, 
                        features = list(tmp_markers$mouse_symbol),
                        name = paste(type, "enriched", sep = "_"))
}

FeaturePlot(seu, features = c("Distal.tubule.cells_enriched", 
                              "Intercalated.cells_enriched", 
                              "Juxtaglomerular.cells_enriched", 
                              "Podocytes_enriched", 
                              "Mesangial.cells_enriched", 
                              "Proximal.tubule.cells_enriched", 
                              "Loop.of.Henle.cells_enriched"), 
            cols = c("blue", "red"))

##Get cell type from reference dataset

reference_seu <- readRDS("./data/Seurat Objects/reference_seu.rds")

anchors <- FindTransferAnchors(reference = reference_seu, query = seu,
                               dims = 1:15, reference.reduction = "pca")
predictions <- TransferData(anchorset = anchors, refdata = reference_seu$type,
                            dims = 1:15)
seu <- AddMetaData(seu, metadata = predictions)

DimPlot(seu, reduction = "umap", label = T, group.by="predicted.id", label.size = 7)

##Diagnostic plots
DimPlot(seu, label = T, shuffle = T, repel = T) + NoLegend()
# split by sample
DimPlot(seu, label = T, shuffle = T, repel = T, split.by = "sample") + NoLegend()
# group by sample
DimPlot(seu, label = T, shuffle = T, repel = T, group.by = "sample") + NoLegend()
# QC plots
p1 <- FeaturePlot(seu, features = "nCount_RNA")
p2 <- FeaturePlot(seu, features = "nFeature_RNA")
p3 <- FeaturePlot(seu, features = "percent_mito")
p4 <- FeaturePlot(seu, features = "scDblFinder.score")
cowplot::plot_grid(p1,p2,p3,p4, ncol = 2)
#Barplot of cell types,
seu$celltype <- Idents(seu)# Before that step, set identities to the cell type. 
                           # Alternatively, use the metadata column where the cell types are described.
df <- as.data.frame(table(seu$celltype))

ggplot(df, aes(Var1, Freq, fill = Var2)) + 
  geom_bar(stat = "identity")  + 
  theme_minimal() + 
  theme(axis.text.x = element_text(angle = 90, size = 15)) +
  theme(axis.text.y = element_text(size = 15)) +
  theme(axis.title = element_text(size = 15)) +
  theme(legend.text = element_text(size = 15)) +
  theme(title = element_text(size = 15)) +
  theme(legend.title=element_blank()) +
  xlab("Celltype") + 
  ylab("Count") +
  ggtitle("Celltype distribution") +
  scale_fill_viridis_d()

##Step 8: Differential expression
#FindMarkers can be used for differential expression analysis. 
#There are many available methods, but for a first approach, the default can be used.
#Specific cell types can be subset and DE analysis conducted within them.

# Define function for DE within subset (in this example, mut vs all wt. The gtype metadata column contains this info.)
FindMarkers_in_subset <- function(x, y){
  tmp_subset <- subset(x, subset = gtype == "wt" | (gtype == "mut" & age == y))
  
  Idents(tmp_subset) <- tmp_subset$gtype
  tmp_DE_table <- FindMarkers(tmp_subset, 
                              ident.1 = "mut",
                              ident.2 = "wt",
                              verbose = T,
                              #min.cells.group = 50)
                              min.cells.group = 10,
                              min.pct = 0.1, 
                              logfc.threshold = -Inf,
                              min.cells.feature = 1)
  tmp_DE_table$gene <- rownames(tmp_DE_table)
  tmp_DE_table$age <- y
  return(tmp_DE_table)
}

### Endothelial cell markers in different ages (example)
endo_subset <- subset(seu, idents = c("Endothelial"))
Idents(endo_subset) <- endo_subset$celltype_gtype

endo_DE <- data.frame()

for(a in c(4,6,8,12)){
  tmp_DE <- FindMarkers_in_subset(endo_subset, a)
  endo_DE <- rbind.data.frame(endo_DE, tmp_DE)
}

### Volcano Plots for FindMarkers
tmp_obj <- endo_DE
celltype <- "Endothelial"
age = 12

data <- tmp_obj[tmp_obj$age == age,] 
# Label DE genes
data$DE <- "not sig"
data$DE[data$avg_log2FC > 0.6 & data$p_val_adj < 0.01] <- "up" 
data$DE[data$avg_log2FC < -0.6 & data$p_val_adj < 0.01 ] <- "down"
data$DElabel <- NA

# Get top 10 regulated genes for labeling
top10 <- head(arrange(data, p_val_adj), 10)

# Take the highest p value of the top10 to set the threshold for labeling
thr <- top10$p_val_adj[10]

# Change labels
data$DElabel[data$p_val_adj <= thr & !is.na(data$p_val_adj) & abs(data$avg_log2FC) > 0.6] <- data$gene[data$p_val_adj <= thr & !is.na(data$p_val_adj) & abs(data$avg_log2FC) > 0.6]

# Volcano plot
ggplot(data, aes(x = avg_log2FC, y = -log(p_val_adj), col = DE, label = DElabel)) +
  geom_point(size = 4) +
  theme_classic() + 
  theme(legend.position = "none") +
  geom_label_repel(colour = "black", max.overlaps = 10, size = 5) +
  scale_color_manual(values = c("blue4", "grey", "red")) +
  ggtitle(paste(celltype, age, "weeks", sep = " ")) +
  theme(text = element_text(size = 20), plot.title = element_text(hjust = 0.5), legend.position = "right")

##Pseudobulk differential expression.
#Especially useful if there are batch effects that should be modelled.

DefaultAssay(seu) # The default assay should be RNA. scTransform residuals violate the assumptions of the methods
                  # and will return many false positives.

## Define Function
# Define pseudobulk function that returns the DE analysis tables
pseudobulk <- function(seu, weeks, celltype){ #Age and cell type are the factors of interest here, but they can
                                              #be whatever is appropriate for the dataset.
  seu_DE <- seu
  # Subset seurat for condition of interest
  seu_DE <- seu_DE %>%
    subset(subset = gtype == "wt" | age == weeks) 
  
  # Get counts for each celltype in each sample
  counts_list <- AggregateExpression(seu_DE, 
                                     group.by = c("celltype", "sample"),
                                     assays = "RNA",
                                     slot = "counts",
                                     return.seurat = F)
  
  counts <- counts_list$RNA
  # Transpose and create data frame
  counts.t <- as.data.frame(t(counts))
  # Create a list for each celltype 
  splitRows <- gsub("_.*", "", rownames(counts.t))
  # Split dataframe
  counts.split <- split.data.frame(counts.t, 
                                   f = factor(splitRows))
  counts.split.mod <- lapply(counts.split, function(x){
    rownames(x) <- gsub(".*_(.*)", "\\1", rownames(x))
    t(x)
  })
  
#Pseudobulk DE. DESeq2 is used here, but edgeR or limma/voom are also alternatives.
  
  # Get count matrix of cell type
  analysis.table <- NULL
  counts_DE <- counts.split.mod[[celltype]]
  
  #Generate sample level metadata
  colData <- data.frame(samples = colnames(counts_DE))
  colData <- colData %>%
    mutate(genotype = ifelse(grepl("mut", samples), "mut", "wt")) %>%
    column_to_rownames(var = "samples")
  
  ´#Create DESeq2 object
  dds <- DESeqDataSetFromMatrix(countData = counts_DE, 
                                colData = colData,
                                design = ~ genotype)
  
  #Filter: keep genes that have at least 10 reads in total
  keep <- rowSums(counts(dds)) >= 10
  dds <- dds[keep,]
  
  #Run DESEq
  dds <- DESeq(dds)
  
  # Check coefficients for the comparison
  resultsNames(dds)
  
  # Create results object
  res <- results(dds, name = "genotype_mut_vs_wt")
  res
  
  analysis.table <- as.data.frame(dplyr::arrange(as.data.frame(res), padj))
  analysis.table$gene <- rownames(analysis.table)
  analysis.table$age <- weeks
  analysis.table$celltype <- celltype
  return(analysis.table)
}


# Define ages
age_of_interest <- c(4,6,8,12)
# Define cell types
celltypes_of_interest <- c("Endothelial", "Mesangial", "Immune", "Podocytes")
# create empty analysis table list
analysis_table_list <- list()

for(c in celltypes_of_interest){
  print(c)
  tmp_df <- data.frame()
  for(a in age_of_interest){
    print(paste(c, a, "weeks", sep = " "))
    tmp_analysis_table <- pseudobulk(seu, a, c)
    tmp_df <- rbind(tmp_df, tmp_analysis_table)
  }
  analysis_table_list[[c]] <- tmp_df
  gc()
}

#Similarly to FindMarkers, volcano plots can be created to check differentially expressed genes.
